<a href="https://colab.research.google.com/github/daiwikpal/NLP/blob/main/CS4650_Project_2_sp25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Licensing Information:  You are free to use or extend this project for
# educational purposes provided that (1) you do not distribute or publish
# solutions, (2) you retain this notice, and (3) you provide clear
# attribution to The Georgia Institute of Technology, including a link to https://aritter.github.io/CS-7650/
# Attribution Information: This assignment was developed at The Georgia Institute of Technology
# by Alan Ritter (alan.ritter@cc.gatech.edu)

# 📘 Named Entity Recognition (NLP Project 2)
**Course: CS 4650 "Natural Language Processing"**  
**Institution: Georgia Institute of Technology**  
**Semester: Spring 2025**  
**Instructor: Dr. Wei Xu**  
**Teaching Assistants: Tarek Naous, Jonathan Zheng, Xiaofeng Wu, Yao Dou **

---

## 🚀 Introduction
In this assignment, you will implement a bidirectional LSTM-CNN-CRF for sequence labeling, following [this paper by Xuezhe Ma and Ed Hovy](https://www.aclweb.org/anthology/P16-1101.pdf), on the CoNLL named entity recognition dataset.  Before starting the assignment, we recommend reading the Ma and Hovy paper. It is quite helpful to understanding the big picture of what exactly you are going to be implementing.

---

## 💻 Utilizing GPUs
For enhanced training performance, consider utilizing GPUs. To switch your instance to use a GPU, navigate to: Runtime -> Change runtime type -> Hardware accelerator.

---

## 🕯️ Getting Started with PyTorch
This project is centered around PyTorch, a powerful library for building neural networks. If you're new to PyTorch or need a quick refresher, the following resources are highly recommended:
- **Introduction to PyTorch**: Explore these informative [slides](https://cocoxu.github.io/CS4650_spring2025/slides/PyTorch_tutorial.pdf) for a comprehensive introduction.
- **PyTorch Basics**: Gain hands-on experience with this interactive [notebook](http://bit.ly/pytorchbasics).
- **NLP Task Example**: Understand PyTorch's application in NLP through this [text sentiment analysis notebook](http://bit.ly/pytorchexample).

---

**As a first step, ensure you have a personal copy of this notebook. You can do so by downloading it to your local drive (File -> Download -> Download .ipynb) or by saving a copy to your Google Drive (File -> Save a copy in Drive).**


## Imports + GPU

First, let's import some libraries and make sure the runtime has access to a GPU.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Thu Feb 20 22:29:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Download the Data

Run the following code to download the English part of the CoNLL 2003 dataset, the evaluation script and pre-filtered GloVe embeddings we are providing for this data.

In [3]:
#CoNLL 2003 data
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testb
!cat eng.train | awk '{print $1 "\t" $4}' > train
!cat eng.testa | awk '{print $1 "\t" $4}' > dev
!cat eng.testb | awk '{print $1 "\t" $4}' > test

#Evaluation Script
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl

#Pre-filtered GloVe embeddings
!wget https://raw.githubusercontent.com/aritter/aritter.github.io/master/files/glove.840B.300d.conll_filtered.txt

--2025-02-20 22:29:12--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3283420 (3.1M) [text/plain]
Saving to: ‘eng.train’

eng.train           100%[===================>]   3.13M  --.-KB/s    in 0.05s   

2025-02-20 22:29:12 (65.4 MB/s) - ‘eng.train’ saved [3283420/3283420]

--2025-02-20 22:29:12--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## CoNLL Data Format

Run the following cell to see a sample of the data in CoNLL format.  As you can see, each line in the file represents a word and its labeled named entity tag in BIO format.  A blank line is used to seperate sentences.

In [5]:
!head -n 20 train

-DOCSTART-	O
	
EU	I-ORG
rejects	O
German	I-MISC
call	O
to	O
boycott	O
British	I-MISC
lamb	O
.	O
	
Peter	I-PER
Blackburn	I-PER
	
BRUSSELS	I-LOC
1996-08-22	O
	
The	O
European	I-ORG


## Data Preprocessing Diagram

Here is a simplified diagram showing what is happening in the data processing sections coming up. Make sure to read through these sections, as you will need to use these functions in your code!

![](https://drive.google.com/uc?export=view&id=13yYcgvEyLC0m3RZegH8V6fErf-8DRtLd)

## Reading in the Data

Below we proivide a bit of code to read in data in the CoNLL format.  This also reads in the filtered GloVe embeddings, to save you some effort - we will discuss this more later.

In [6]:
#Read in the training data
def read_conll_format(filename):
    (words, tags, currentSent, currentTags) = ([],[],['-START-'],['START'])
    for line in open(filename).readlines():
        line = line.strip()
        #print(line)
        if line == "":
            currentSent.append('-END-')
            currentTags.append('END')
            words.append(currentSent)
            tags.append(currentTags)
            (currentSent, currentTags) = (['-START-'], ['START'])
        else:
            (word, tag) = line.split()
            currentSent.append(word)
            currentTags.append(tag)
    return (words, tags)

def sentences2char(sentences):
    return [[['start'] + [c for c in w] + ['end'] for w in l] for l in sentences]


(sentences_train, tags_train) = read_conll_format("train")
(sentences_dev, tags_dev)     = read_conll_format("dev")

print("The second sentence in train set:", sentences_train[2])
print("The NER label of the sentence:   ", tags_train[2])

sentencesChar = sentences2char(sentences_train)

print("The char repersentation of the sentence:", sentencesChar[2])

The second sentence in train set: ['-START-', 'Peter', 'Blackburn', '-END-']
The NER label of the sentence:    ['START', 'I-PER', 'I-PER', 'END']
The char repersentation of the sentence: [['start', '-', 'S', 'T', 'A', 'R', 'T', '-', 'end'], ['start', 'P', 'e', 't', 'e', 'r', 'end'], ['start', 'B', 'l', 'a', 'c', 'k', 'b', 'u', 'r', 'n', 'end'], ['start', '-', 'E', 'N', 'D', '-', 'end']]


In [7]:
#Read GloVe embeddings.
def read_GloVe(filename):
    embeddings = {}
    for line in open(filename).readlines():
        #print(line)
        fields = line.strip().split(" ")
        word = fields[0]
        embeddings[word] = [float(x) for x in fields[1:]]
    return embeddings

GloVe = read_GloVe("glove.840B.300d.conll_filtered.txt")

print("The GloVe word embedding of the word 'the':", GloVe["the"])
print("dimension of glove embedding:", len(GloVe["the"]))

The GloVe word embedding of the word 'the': [0.27204, -0.06203, -0.1884, 0.023225, -0.018158, 0.0067192, -0.13877, 0.17708, 0.17709, 2.5882, -0.35179, -0.17312, 0.43285, -0.10708, 0.15006, -0.19982, -0.19093, 1.1871, -0.16207, -0.23538, 0.003664, -0.19156, -0.085662, 0.039199, -0.066449, -0.04209, -0.19122, 0.011679, -0.37138, 0.21886, 0.0011423, 0.4319, -0.14205, 0.38059, 0.30654, 0.020167, -0.18316, -0.0065186, -0.0080549, -0.12063, 0.027507, 0.29839, -0.22896, -0.22882, 0.14671, -0.076301, -0.1268, -0.0066651, -0.052795, 0.14258, 0.1561, 0.05551, -0.16149, 0.09629, -0.076533, -0.049971, -0.010195, -0.047641, -0.16679, -0.2394, 0.0050141, -0.049175, 0.013338, 0.41923, -0.10104, 0.015111, -0.077706, -0.13471, 0.119, 0.10802, 0.21061, -0.051904, 0.18527, 0.17856, 0.041293, -0.014385, -0.082567, -0.035483, -0.076173, -0.045367, 0.089281, 0.33672, -0.22099, -0.0067275, 0.23983, -0.23147, -0.88592, 0.091297, -0.012123, 0.013233, -0.25799, -0.02972, 0.016754, 0.01369, 0.32377, 0.039546, 0.

## Mapping Tokens to Indices

As in the last project, we will need to convert words in the dataset to numeric indices, so they can be presented as input to a neural network.  Code to handle this for you with sample usage is provided below.

In [8]:
#Create mappings between tokens and indices.

from collections import Counter
import random

#Will need this later to remove 50% of words that only appear once in the training data from the vocabulary (and don't have GloVe embeddings).
wordCounts = Counter([w for l in sentences_train for w in l])
charCounts = Counter([c for l in sentences_train for w in l for c in w])
singletons = set([w for (w,c) in wordCounts.items() if c == 1 and not w in GloVe.keys()])
charSingletons = set([w for (w,c) in charCounts.items() if c == 1])

#Build dictionaries to map from words, characters to indices and vice versa.
#Save first two words in the vocabulary for padding and "UNK" token.
word2i = {w:i+2 for i,w in enumerate(set([w for l in sentences_train for w in l] + list(GloVe.keys())))}
char2i = {w:i+2 for i,w in enumerate(set([c for l in sentencesChar for w in l for c in w]))}
i2word = {i:w for w,i in word2i.items()}
i2char = {i:w for w,i in char2i.items()}

vocab_size = max(word2i.values()) + 1
char_vocab_size = max(char2i.values()) + 1

#Tag dictionaries.
tag2i = {w:i for i,w in enumerate(set([t for l in tags_train for t in l]))}
i2tag = {i:t for t,i in tag2i.items()}

#When training, randomly replace singletons with UNK tokens sometimes to simulate situation at test time.
def getDictionaryRandomUnk(w, dictionary, train=False):
    if train and (w in singletons and random.random() > 0.5):
        return 1
    else:
        return dictionary.get(w, 1)

#Map a list of sentences from words to indices.
def sentences2indices(words, dictionary, train=False):
    #1.0 => UNK
    return [[getDictionaryRandomUnk(w,dictionary, train=train) for w in l] for l in words]

#Map a list of sentences containing to indices (character indices)
def sentences2indicesChar(chars, dictionary):
    #1.0 => UNK
    return [[[dictionary.get(c,1) for c in w] for w in l] for l in chars]

#Indices
X       = sentences2indices(sentences_train, word2i, train=True)
X_char  = sentences2indicesChar(sentencesChar, char2i)
Y       = sentences2indices(tags_train, tag2i)

print("vocab size:", vocab_size)
print("char vocab size:", char_vocab_size)
print()

print("index of word 'the':", word2i["the"])
print("word of index 253:", i2word[253])
print()

#Print out some examples of what the dev inputs will look like
for i in range(10):
    print(" ".join([i2word.get(w,'UNK') for w in X[i]]))

vocab size: 29148
char vocab size: 88

index of word 'the': 19425
word of index 253: tent

-START- -DOCSTART- -END-
-START- EU rejects German call to boycott British lamb . -END-
-START- Peter Blackburn -END-
-START- BRUSSELS 1996-08-22 -END-
-START- The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep . -END-
-START- Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . -END-
-START- " We do n't support any such recommendation because we do n't see any grounds for it , " the Commission 's chief spokesman Nikolaus van der Pas told a news briefing . -END-
-START- He said further scientific study was required and if it was found that action was needed it should be taken by the European Union . -END-


## Padding and Batching

In this assignment, you should train your models using minibatched SGD.  When presenting multiple sentences to the network at the same time, we will need to pad them to be of the same length. We use [torch.nn.utils.rnn.pad_sequence](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html) to do so.


Below we provide some code to prepare batches of data to present to the network. We pad the sequence so that all the sequences have the same length.

**Side Note:** PyTorch includes utilities in [`torch.utils.data`](https://pytorch.org/docs/stable/data.html) to help with padding, batching, shuffling and some other things, but for this assignment we will do everything from scratch to help you see exactly how this works.

In [9]:
#Pad inputs to max sequence length (for batching)
def prepare_input(X_list):
    X_padded = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(l) for l in X_list], batch_first=True).type(torch.LongTensor) # padding the sequences with 0
    X_mask   = torch.nn.utils.rnn.pad_sequence([torch.as_tensor([1.0] * len(l)) for l in X_list], batch_first=True).type(torch.FloatTensor) # consisting of 0 and 1, 0 for padded positions, 1 for non-padded positions
    return (X_padded, X_mask)

#Maximum word length (for character representations)
MAX_CLEN = 32

def prepare_input_char(X_list):
    MAX_SLEN = max([len(l) for l in X_list])
    X_padded  = [l + [[]]*(MAX_SLEN-len(l))  for l in X_list]
    X_padded  = [[w[0:MAX_CLEN] for w in l] for l in X_padded]
    X_padded  = [[w + [1]*(MAX_CLEN-len(w)) for w in l] for l in X_padded]
    return torch.as_tensor(X_padded).type(torch.LongTensor)

#Pad outputs using one-hot encoding
def prepare_output_onehot(Y_list, NUM_TAGS=max(tag2i.values())+1):
    Y_onehot = [torch.zeros(len(l), NUM_TAGS) for l in Y_list]
    for i in range(len(Y_list)):
        for j in range(len(Y_list[i])):
            # print(type(Y_list[i][j]))
            Y_onehot[i][j,Y_list[i][j]] = 1.0
    Y_padded = torch.nn.utils.rnn.pad_sequence(Y_onehot, batch_first=True).type(torch.FloatTensor)
    return Y_padded

print("max slen:", max([len(x) for x in X_char]))

print(Y)
(X_padded, X_mask) = prepare_input(X)
X_padded_char      = prepare_input_char(X_char)
Y_onehot           = prepare_output_onehot(Y)

print("X_padded:", X_padded.shape)
print("X_mask:", X_mask.shape)
print("X_padded_char:", X_padded_char.shape)
print("Y_onehot:", Y_onehot.shape)

max slen: 115
[[4, 7, 0], [4, 5, 7, 8, 7, 7, 7, 8, 7, 7, 0], [4, 1, 1, 0], [4, 9, 7, 0], [4, 7, 5, 5, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0], [4, 9, 7, 7, 7, 7, 5, 5, 7, 7, 7, 1, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 9, 7, 7, 7, 7, 7, 7, 7, 0], [4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 5, 7, 7, 7, 1, 1, 1, 1, 7, 7, 7, 7, 7, 0], [4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 5, 5, 7, 0], [4, 7, 7, 7, 7, 7, 7, 7, 5, 7, 7, 1, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0], [4, 1, 7, 8, 7, 7, 7, 7, 9, 7, 9, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 7, 8, 7, 7, 7, 7, 7, 7, 0], [4, 7, 1, 7, 7, 7, 7, 7, 7, 7, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0], [4, 8, 7, 7, 1, 1, 1, 7, 7, 7, 1, 7, 7, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0], [4, 7, 0], [4, 7, 9, 7, 9, 7, 1, 7, 7, 7, 0], [4, 7, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,

## Model Diagram

Below is a simplified diagram of the model you will be implementing. A lot is abstracted away here, but it should help you get the bigger picture of the model you are implementing. Red is part 1, green is part 2, and purple is the extra credit, part 3.


![](https://drive.google.com/uc?export=view&id=1GEkDtTxu-x060pltpiHi5LtcaDpLJdw3)

## **Your code starts here:** Basic LSTM Tagger (10 points)

OK, now you should have everything you need to get started.

Recall that your goal is to to implement the BiLSTM-CNN-CRF, as described in [(Ma and Hovy, 2016)](https://www.aclweb.org/anthology/P16-1101.pdf).  This is a relatively complex network with various components.  Below we provide starter code to break down your implementation into increasingly complex versions of the final model, starting with a Basic LSTM tagger.  This way you can be confident that each part is working correctly before incrementally increasing the complexity of your implementation.  This is generally a good approach to take when implementing complex models, since buggy PyTorch code is often partially working, but produces worse results than a correct implementation, so it's hard to know whether added complexities are helping or hurting.  Also, if you aren't able to match published results it's hard to know which component of your model has the problem (or even whether or not it is a problem in the published result!)

**Fill in the functions marked as `TODO` in the code block below. Please make your code changes only within the given commented block (#####).** If everything is working correctly, you should be able to achieve an **F1 score of 0.86 on the dev set and 0.82 on the test set (with GloVe embeddings)**. You are required to initialize word embeddings with GloVe later, but you can randomly initialize the word embeddings in the beginning.

In [22]:
#####################################################################################
#TODO: Add imports if needed:

#####################################################################################
seed = 42
torch.manual_seed(seed)
random.seed(seed)

class BasicLSTMtagger(nn.Module):
    def __init__(self, DIM_EMB=10, DIM_HID=10):
        super(BasicLSTMtagger, self).__init__()
        NUM_TAGS = max(tag2i.values())+1


        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
        #####################################################################################
        #TODO: initialize parameters - embedding layer, nn.LSTM, nn.Linear and nn.LogSoftmax

        self.vocab_size = max(word2i.values())+1

        self.word_embeddings = nn.Embedding(vocab_size, self.DIM_EMB)
        self.init_glove(GloVe)

        self.lstm = nn.LSTM(self.DIM_EMB, DIM_HID, bidirectional=True, batch_first=True)

        self.linear = nn.Linear(2 * DIM_HID, self.NUM_TAGS)

        self.LogSoftmax = nn.LogSoftmax(dim = 2)

        #####################################################################################

    def forward(self, X, train=False):
        #####################################################################################
        #TODO: Implement the forward computation.

        embed = self.word_embeddings(X)

        lstm_out, _ = self.lstm(embed)

        linear_out = self.linear(lstm_out)

        output = self.LogSoftmax(linear_out)

        return output


        #####################################################################################

    def init_glove(self, GloVe):
        #####################################################################################
        #TODO: initialize word embeddings using GloVe (you can skip this part in your first version, if you want, see instructions below).


        embedding_matrix = torch.FloatTensor(self.vocab_size, self.DIM_EMB).uniform_(-0.1, 0.1)

        for word, idx in word2i.items():
          if word in GloVe:
              # print(type(torch.tensor(GloVe[word], dtype=torch.float32)))

              embedding_matrix[idx] = torch.tensor(GloVe[word], dtype=torch.float32)

        self.word_embeddings.weight.data.copy_(embedding_matrix)
        self.word_embeddings.weight.requires_grad = False

        #####################################################################################

    def inference(self, sentences):
        X, X_mask       = prepare_input(sentences2indices(sentences, word2i))
        pred = self.forward(X.cuda()).argmax(dim=2)
        return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
        Y_pred = self.inference(words)
        for i in range(len(words)):
            print("----------------------------")
            print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
            print("Predicted:\t", Y_pred[i])
            print("Gold:\t\t", tags[i])

    def write_predictions(self, sentences, outFile):
        fOut = open(outFile, 'w')
        for s in sentences:
            y = self.inference([s])[0]
            #print("\n".join(y[1:len(y)-1]))
            fOut.write("\n".join(y[1:len(y)-1]))  #Skip start and end tokens
            fOut.write("\n\n")

#The following code will initialize a model and test that your forward computation runs without errors.
lstm_test   = BasicLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output = lstm_test.forward(prepare_input(X[0:5])[0])
Y_onehot    = prepare_output_onehot(Y[0:5])

#Check the shape of the lstm_output and one-hot label tensors.
print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


In [20]:
#Read in the data

(sentences_dev, tags_dev)     = read_conll_format('dev')
(sentences_train, tags_train) = read_conll_format('train')
(sentences_test, tags_test)   = read_conll_format('test')

## Train your Model (10 points)

Next, implement the function below to train your basic BiLSTM tagger.  See [torch.nn.lstm](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html).  Make sure to save your predictions on the test set (`test_pred_lstm.txt`) for submission to GradeScope. Feel free to change number of epochs, optimizer, learning rate and batch size.

In [24]:
#Training

#####################################################################################
#TODO: Add imports if needed:

from torch.optim import Adam, AdamW, SGD


#####################################################################################

from random import sample
from tqdm import tqdm
import os
import subprocess
import random

def shuffle_sentences(sentences, tags):
    shuffled_sentences = []
    shuffled_tags      = []
    indices = list(range(len(sentences)))
    random.shuffle(indices)
    for i in indices:
        shuffled_sentences.append(sentences[i])
        shuffled_tags.append(tags[i])
    return (shuffled_sentences, shuffled_tags)


def train_basic_lstm(sentences, tags, lstm):
    #####################################################################################
    #TODO: initialize optimizer and other parameters

    optim = AdamW(lstm.parameters(), lr=0.001)
    nEpochs = 15
    batchSize = 64
    loss_function = nn.NLLLoss()

    lstm.train()
    lstm.cuda()


    #####################################################################################

    for epoch in range(nEpochs):
        totalLoss = 0.0

        (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
        for batch in tqdm(range(0, len(sentences), batchSize), leave=False):
            #####################################################################################
            #TODO: Implement gradient update on a batch of data.

            optim.zero_grad()

            input_data = prepare_input(sentences2indices(sentences_shuffled[batch:batch+batchSize], word2i, train=True))[0].cuda()

            lstm_output = lstm.forward(input_data)

            Y_onehot = prepare_output_onehot(sentences2indices(tags_shuffled[batch:batch+batchSize], tag2i)).cuda()

            loss = loss_function(lstm_output.view(-1, lstm.NUM_TAGS), Y_onehot.view(-1, lstm.NUM_TAGS).argmax(dim=1))

            loss.backward()

            optim.step()

            totalLoss += loss

            #####################################################################################

        print(f"loss on epoch {epoch} = {totalLoss}")
        lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
        print('conlleval:')
        print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

        if epoch % 10 == 0:
            s = sample(range(len(sentences_dev)), 5)
            lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])


lstm = BasicLSTMtagger(DIM_HID=500, DIM_EMB=300).cuda()
train_basic_lstm(sentences_train, tags_train, lstm)

loss on epoch 0 = 31.517009735107422
conlleval:
processed 51578 tokens with 5942 phrases; found: 5581 phrases; correct: 4436.
accuracy:  95.77%; precision:  79.48%; recall:  74.65%; FB1:  76.99
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  27
              LOC: precision:  86.91%; recall:  79.15%; FB1:  82.85  1673
             MISC: precision:  74.49%; recall:  63.34%; FB1:  68.46  784
              ORG: precision:  66.19%; recall:  59.13%; FB1:  62.47  1198
              PER: precision:  84.52%; recall:  87.13%; FB1:  85.81  1899

----------------------------
-START-/START/START -DOCSTART-/O/O -END-/END/END
Predicted:	 ['START', 'O', 'END']
Gold:		 ['START', 'O', 'END']
----------------------------
-START-/START/START BASEBALL/O/O -/O/O MAJOR/O/I-MISC LEAGUE/O/I-MISC RESULTS/O/O THURSDAY/O/O ./O/O -END-/END/END
Predicted:	 ['START', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'END']
Gold:		 ['START', 'O', 'O', 'I-MISC', 'I-MISC', 'O', 'O', 'O', 'END']
------------------

loss on epoch 1 = 6.205544471740723
conlleval:
processed 51578 tokens with 5942 phrases; found: 6008 phrases; correct: 5137.
accuracy:  97.62%; precision:  85.50%; recall:  86.45%; FB1:  85.97
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  12
              LOC: precision:  89.54%; recall:  91.83%; FB1:  90.67  1884
             MISC: precision:  77.92%; recall:  78.85%; FB1:  78.38  933
              ORG: precision:  78.34%; recall:  76.88%; FB1:  77.61  1316
              PER: precision:  90.82%; recall:  91.86%; FB1:  91.34  1863



loss on epoch 2 = 4.261541366577148
conlleval:
processed 51578 tokens with 5942 phrases; found: 6016 phrases; correct: 5264.
accuracy:  98.05%; precision:  87.50%; recall:  88.59%; FB1:  88.04
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  4
              LOC: precision:  91.82%; recall:  91.62%; FB1:  91.72  1833
             MISC: precision:  80.83%; recall:  80.48%; FB1:  80.65  918
              ORG: precision:  78.56%; recall:  83.89%; FB1:  81.14  1432
              PER: precision:  93.71%; recall:  93.05%; FB1:  93.38  1829



loss on epoch 3 = 3.1860687732696533
conlleval:
processed 51578 tokens with 5942 phrases; found: 5959 phrases; correct: 5316.
accuracy:  98.20%; precision:  89.21%; recall:  89.46%; FB1:  89.34
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  2
              LOC: precision:  94.41%; recall:  91.02%; FB1:  92.68  1771
             MISC: precision:  84.75%; recall:  81.34%; FB1:  83.01  885
              ORG: precision:  80.48%; recall:  85.76%; FB1:  83.03  1429
              PER: precision:  93.16%; recall:  94.68%; FB1:  93.91  1872



loss on epoch 4 = 2.3049581050872803
conlleval:
processed 51578 tokens with 5942 phrases; found: 6032 phrases; correct: 5391.
accuracy:  98.40%; precision:  89.37%; recall:  90.73%; FB1:  90.05
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  2
              LOC: precision:  92.77%; recall:  94.99%; FB1:  93.87  1881
             MISC: precision:  84.38%; recall:  82.00%; FB1:  83.17  896
              ORG: precision:  83.96%; recall:  85.46%; FB1:  84.70  1365
              PER: precision:  92.37%; recall:  94.68%; FB1:  93.51  1888



...

loss on epoch 10 = 0.22890008985996246
conlleval:
processed 51578 tokens with 5942 phrases; found: 6008 phrases; correct: 5424.
accuracy:  98.53%; precision:  90.28%; recall:  91.28%; FB1:  90.78
              LOC: precision:  92.44%; recall:  95.92%; FB1:  94.15  1906
             MISC: precision:  85.43%; recall:  83.30%; FB1:  84.35  899
              ORG: precision:  86.88%; recall:  84.94%; FB1:  85.90  1311
              PER: precision:  92.76%; recall:  95.28%; FB1:  94.00  1892

----------------------------
-START-/START/START Everything/O/O else/O/O is/O/O muddy/O/O ,/O/O the/O/O waters/O/O of/O/O the/O/O fjord/O/O leaden/O/O ./O/O -END-/END/END
Predicted:	 ['START', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'END']
Gold:		 ['START', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'END']
----------------------------
-START-/START/START Most/O/O of/O/O the/O/O Marines/I-ORG/I-MISC are/O/O on/O/O three/O/O ships/O/O in/O/O the/O/O Tarawa/I-ORG/I-ORG 

loss on epoch 11 = 0.16334421932697296
conlleval:
processed 51578 tokens with 5942 phrases; found: 6062 phrases; correct: 5455.
accuracy:  98.56%; precision:  89.99%; recall:  91.80%; FB1:  90.89
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  2
              LOC: precision:  94.19%; recall:  94.50%; FB1:  94.35  1843
             MISC: precision:  84.98%; recall:  84.71%; FB1:  84.85  919
              ORG: precision:  83.40%; recall:  87.70%; FB1:  85.50  1410
              PER: precision:  93.33%; recall:  95.66%; FB1:  94.48  1888



loss on epoch 12 = 0.14687789976596832
conlleval:
processed 51578 tokens with 5942 phrases; found: 6056 phrases; correct: 5453.
accuracy:  98.53%; precision:  90.04%; recall:  91.77%; FB1:  90.90
              LOC: precision:  94.26%; recall:  95.59%; FB1:  94.92  1863
             MISC: precision:  83.69%; recall:  84.06%; FB1:  83.87  926
              ORG: precision:  84.51%; recall:  86.65%; FB1:  85.57  1375
              PER: precision:  93.02%; recall:  95.55%; FB1:  94.27  1892



loss on epoch 13 = 0.12515994906425476
conlleval:
processed 51578 tokens with 5942 phrases; found: 6037 phrases; correct: 5434.
accuracy:  98.54%; precision:  90.01%; recall:  91.45%; FB1:  90.73
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  2
              LOC: precision:  93.90%; recall:  94.67%; FB1:  94.28  1852
             MISC: precision:  85.84%; recall:  83.51%; FB1:  84.66  897
              ORG: precision:  83.29%; recall:  87.32%; FB1:  85.26  1406
              PER: precision:  93.30%; recall:  95.22%; FB1:  94.25  1880



loss on epoch 14 = 0.0909266471862793
conlleval:
processed 51578 tokens with 5942 phrases; found: 6010 phrases; correct: 5459.
accuracy:  98.58%; precision:  90.83%; recall:  91.87%; FB1:  91.35
              LOC: precision:  94.30%; recall:  95.43%; FB1:  94.86  1859
             MISC: precision:  86.44%; recall:  84.38%; FB1:  85.40  900
              ORG: precision:  85.41%; recall:  86.43%; FB1:  85.92  1357
              PER: precision:  93.40%; recall:  96.04%; FB1:  94.70  1894



In [25]:
#Evaluation on test data
lstm.write_predictions(sentences_test, 'test_pred_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_lstm.txt | perl conlleval.pl -d "\t"

--2025-02-20 22:52:00--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.5’

conlleval.pl.5      100%[===================>]  12.46K  --.-KB/s    in 0.001s  

2025-02-20 22:52:00 (18.3 MB/s) - ‘conlleval.pl.5’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5732 phrases; correct: 4926.
accuracy:  97.49%; precision:  85.94%; recall:  87.22%; FB1:  86.57
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  88.69%; recall:  91.19%; FB1:  89.92  1715
             MISC: precision:  73.46%; recall:  74.93%; FB1:  74.19  716
              ORG: precision

## Initialization with GloVe Embeddings (5 points)

If you haven't already, implement the `init_glove()` method in `BasicLSTMtagger` above.

Rather than initializing word embeddings randomly, it is common to use learned word embeddings (GloVe or Word2Vec), as discussed in lecture.  To make this simpler, we have already pre-filtered [GloVe](https://nlp.stanford.edu/projects/glove/) embeddings to only contain words in the vocabulary of the CoNLL NER dataset, and loaded them into a dictionary (`GloVe`) at the beginning of this notebook.



## Character Embeddings (10 points)

Now that you have your basic LSTM tagger working, the next step is to add a convolutional network that computes word embeddings from character representations of words.  See Figure 2 and Figure 3 in the [Ma and Hovy](https://www.aclweb.org/anthology/P16-1101.pdf) paper.  We have provided code in `sentences2input_tensors` to convert sentences into lists of word and character indices.  See also [nn.Conv1d](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html) and [MaxPool1d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html).

Hint: The nn.Conv1d accepts input size $(N, C_{in}, L_{in})$, but we have input size $(N, \text{SLEN}, \text{CLEN}, \text{EMB_DIM})$. We can reshape and [permute](https://pytorch.org/docs/stable/generated/torch.permute.html) our input to satisfy the nn.Conv1d, and recover the dimensions later.

Make sure to save your predictions on the test set, for submission to GradeScope. You should be able to achieve **90 F1 / 85 F1 on the dev/test sets**.

**Fill in the functions marked as `TODO` in the code block below. Please make your code changes only within the given commented block (#####).**

In [58]:
#####################################################################################
#TODO: Add imports if needed:
import torch.nn.functional as F
#####################################################################################


class CharLSTMtagger(BasicLSTMtagger):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10):
        super(CharLSTMtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID)
        NUM_TAGS = max(tag2i.values())+1

        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
        #####################################################################################
        #TODO: Initialize parameters.


        self.char_embeddings = nn.Embedding(char_vocab_size, DIM_CHAR_EMB)
        self.char_cnn = nn.Conv1d(in_channels = DIM_CHAR_EMB, out_channels = 30, kernel_size = 3, padding = 1)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        self.char_dropout = nn.Dropout(p=0.5)
        self.embeddings_words = nn.Embedding(vocab_size, DIM_EMB)
        self.dropout_in = nn.Dropout(p=0.5)

        self.log_softmax = nn.LogSoftmax(dim = 2)
        self.linear = nn.Linear(2 * DIM_HID, NUM_TAGS)
        self.lstm = nn.LSTM(DIM_EMB + 30, DIM_HID, batch_first = True, bidirectional = True)
        self.init_glove(GloVe)

        #####################################################################################

    def forward(self, X, X_char, train=False):
        #####################################################################################
        #TODO: Implement the forward computation.

        word_embeddings = self.word_embeddings(X)


        char_embeddings = self.char_embeddings(X_char)
        if train:
            char_embeddings = self.char_dropout(char_embeddings)


        batch_size, seq_len, word_len, char_dim = char_embeddings.shape
        char_embeddings = char_embeddings.view(-1, char_dim, word_len)


        char_cnn_out = F.relu(self.char_cnn(char_embeddings))
        char_pooled = self.maxpool(char_cnn_out)


        char_rep = torch.max(char_pooled, dim=2)[0]
        char_rep = char_rep.view(batch_size, seq_len, -1)


        combined_embeddings = torch.cat((word_embeddings, char_rep), dim=2)

        if train:
            combined_embeddings = self.dropout_in(combined_embeddings)


        lstm_out, _ = self.lstm(combined_embeddings)


        linear_out = self.linear(lstm_out)
        output = self.log_softmax(linear_out)

        return output

        #####################################################################################

    def sentences2input_tensors(self, sentences):
        (X, X_mask)   = prepare_input(sentences2indices(sentences, word2i))
        X_char        = prepare_input_char(sentences2indicesChar(sentences, char2i))
        return (X, X_mask, X_char)

    def inference(self, sentences):
        (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
        pred = self.forward(X.cuda(), X_char.cuda()).argmax(dim=2)
        return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
        Y_pred = self.inference(words)
        for i in range(len(words)):
            print("----------------------------")
            print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
            print("Predicted:\t", Y_pred[i])
            print("Gold:\t\t", tags[i])

char_lstm_test = CharLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output    = char_lstm_test.forward(prepare_input(X[0:5])[0], prepare_input_char(X_char[0:5]))
Y_onehot       = prepare_output_onehot(Y[0:5])

print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


In [62]:
#Training LSTM w/ character embeddings. Feel free to change number of epochs, optimizer, learning rate and batch size.

#####################################################################################
#TODO: Add imports if necessary.

#####################################################################################


def train_char_lstm(sentences, tags, lstm):
    #####################################################################################
    #TODO: initialize optimizer and other hyperparameters.
    optim = AdamW(lstm.parameters(), lr=0.001)
    nEpochs = 10
    batchSize = 138
    loss_function = nn.NLLLoss()

    # lstm.train()
    lstm.cuda()

    #####################################################################################

    for epoch in range(nEpochs):
        totalLoss = 0.0

        (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
        for batch in tqdm(range(0, len(sentences), batchSize), leave=False):

            #####################################################################################
            #TODO: Implement gradient update on a batch of data.

            optim.zero_grad()

            input_data = prepare_input(sentences2indices(sentences_shuffled[batch:batch+batchSize], word2i, train=True))[0].cuda()

            X_char = prepare_input_char(sentences2indicesChar(sentences_shuffled[batch:batch+batchSize], char2i))
            X_char = X_char.cuda()
            lstm_output = lstm.forward(input_data, X_char, train=True)

            Y_onehot = prepare_output_onehot(sentences2indices(tags_shuffled[batch:batch+batchSize], tag2i)).cuda()

            loss = loss_function(lstm_output.view(-1, lstm.NUM_TAGS), Y_onehot.view(-1, lstm.NUM_TAGS).argmax(dim=1))

            loss.backward()

            optim.step()

            totalLoss += loss


            #####################################################################################

        print(f"loss on epoch {epoch} = {totalLoss}")
        lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
        print('conlleval:')
        print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

        if epoch % 10 == 0:
            s = sample(range(len(sentences_dev)), 5)
            lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

char_lstm = CharLSTMtagger(DIM_HID=500, DIM_EMB=300).cuda()
train_char_lstm(sentences_train, tags_train, char_lstm)

loss on epoch 0 = 23.20186996459961
conlleval:
processed 51578 tokens with 5942 phrases; found: 6020 phrases; correct: 4171.
accuracy:  94.46%; precision:  69.29%; recall:  70.20%; FB1:  69.74
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  13
              LOC: precision:  69.39%; recall:  82.20%; FB1:  75.26  2176
             MISC: precision:  64.97%; recall:  60.95%; FB1:  62.90  865
              ORG: precision:  56.60%; recall:  47.95%; FB1:  51.92  1136
              PER: precision:  79.56%; recall:  79.04%; FB1:  79.30  1830

----------------------------
-START-/START/START LONDON/I-LOC/I-LOC 1996-08-31/O/O -END-/END/END
Predicted:	 ['START', 'I-LOC', 'O', 'END']
Gold:		 ['START', 'I-LOC', 'O', 'END']
----------------------------
-START-/START/START Several/O/O other/O/O leading/O/O members/O/O of/O/O the/O/O NLD/O/I-ORG have/O/O served/O/O prison/O/O terms/O/O ./O/O -END-/END/END
Predicted:	 ['START', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

loss on epoch 1 = 5.868657112121582
conlleval:
processed 51578 tokens with 5942 phrases; found: 6129 phrases; correct: 4681.
accuracy:  96.26%; precision:  76.37%; recall:  78.78%; FB1:  77.56
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  5
              LOC: precision:  86.07%; recall:  82.04%; FB1:  84.00  1751
             MISC: precision:  68.68%; recall:  70.17%; FB1:  69.42  942
              ORG: precision:  60.38%; recall:  68.31%; FB1:  64.10  1517
              PER: precision:  84.17%; recall:  87.46%; FB1:  85.78  1914



loss on epoch 2 = 4.648191452026367
conlleval:
processed 51578 tokens with 5942 phrases; found: 5975 phrases; correct: 4881.
accuracy:  97.02%; precision:  81.69%; recall:  82.14%; FB1:  81.92
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  2
              LOC: precision:  88.65%; recall:  88.02%; FB1:  88.34  1824
             MISC: precision:  73.24%; recall:  74.51%; FB1:  73.87  938
              ORG: precision:  70.64%; recall:  67.64%; FB1:  69.10  1284
              PER: precision:  86.66%; recall:  90.66%; FB1:  88.62  1927



loss on epoch 3 = 3.8985445499420166
conlleval:
processed 51578 tokens with 5942 phrases; found: 6021 phrases; correct: 5077.
accuracy:  97.53%; precision:  84.32%; recall:  85.44%; FB1:  84.88
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  2
              LOC: precision:  88.47%; recall:  92.71%; FB1:  90.54  1925
             MISC: precision:  82.03%; recall:  73.75%; FB1:  77.67  829
              ORG: precision:  75.13%; recall:  74.79%; FB1:  74.96  1335
              PER: precision:  87.62%; recall:  91.80%; FB1:  89.66  1930



loss on epoch 4 = 3.2818994522094727
conlleval:
processed 51578 tokens with 5942 phrases; found: 5950 phrases; correct: 5149.
accuracy:  97.84%; precision:  86.54%; recall:  86.65%; FB1:  86.60
              LOC: precision:  92.85%; recall:  90.53%; FB1:  91.68  1791
             MISC: precision:  79.14%; recall:  78.20%; FB1:  78.67  911
              ORG: precision:  79.56%; recall:  78.08%; FB1:  78.81  1316
              PER: precision:  88.92%; recall:  93.27%; FB1:  91.04  1932



loss on epoch 5 = 2.898097515106201
conlleval:
processed 51578 tokens with 5942 phrases; found: 6009 phrases; correct: 5228.
accuracy:  97.94%; precision:  87.00%; recall:  87.98%; FB1:  87.49
              LOC: precision:  91.35%; recall:  93.69%; FB1:  92.50  1884
             MISC: precision:  84.00%; recall:  76.90%; FB1:  80.29  844
              ORG: precision:  76.76%; recall:  80.54%; FB1:  78.60  1407
              PER: precision:  91.68%; recall:  93.27%; FB1:  92.47  1874



loss on epoch 6 = 2.46590518951416
conlleval:
processed 51578 tokens with 5942 phrases; found: 5963 phrases; correct: 5282.
accuracy:  98.17%; precision:  88.58%; recall:  88.89%; FB1:  88.74
              LOC: precision:  90.18%; recall:  95.97%; FB1:  92.99  1955
             MISC: precision:  82.19%; recall:  79.07%; FB1:  80.60  887
              ORG: precision:  83.61%; recall:  78.37%; FB1:  80.91  1257
              PER: precision:  93.29%; recall:  94.41%; FB1:  93.85  1864



loss on epoch 7 = 2.2488696575164795
conlleval:
processed 51578 tokens with 5942 phrases; found: 6026 phrases; correct: 5345.
accuracy:  98.23%; precision:  88.70%; recall:  89.95%; FB1:  89.32
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  94.07%; recall:  94.07%; FB1:  94.07  1837
             MISC: precision:  84.62%; recall:  79.93%; FB1:  82.21  871
              ORG: precision:  79.56%; recall:  84.49%; FB1:  81.95  1424
              PER: precision:  92.29%; recall:  94.84%; FB1:  93.55  1893



loss on epoch 8 = 1.9800550937652588
conlleval:
processed 51578 tokens with 5942 phrases; found: 6025 phrases; correct: 5377.
accuracy:  98.35%; precision:  89.24%; recall:  90.49%; FB1:  89.86
              LOC: precision:  94.40%; recall:  93.69%; FB1:  94.04  1823
             MISC: precision:  85.88%; recall:  79.18%; FB1:  82.39  850
              ORG: precision:  80.26%; recall:  87.92%; FB1:  83.91  1469
              PER: precision:  92.78%; recall:  94.84%; FB1:  93.80  1883



loss on epoch 9 = 1.7647960186004639
conlleval:
processed 51578 tokens with 5942 phrases; found: 6026 phrases; correct: 5412.
accuracy:  98.51%; precision:  89.81%; recall:  91.08%; FB1:  90.44
              LOC: precision:  94.18%; recall:  95.21%; FB1:  94.69  1857
             MISC: precision:  82.39%; recall:  81.67%; FB1:  82.03  914
              ORG: precision:  84.87%; recall:  86.58%; FB1:  85.71  1368
              PER: precision:  92.69%; recall:  94.95%; FB1:  93.81  1887



In [63]:
#Evaluation on test set
char_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm.txt | perl conlleval.pl -d "\t"

--2025-02-21 02:41:03--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.14’

conlleval.pl.14     100%[===================>]  12.46K  --.-KB/s    in 0.001s  

2025-02-21 02:41:03 (20.4 MB/s) - ‘conlleval.pl.14’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5737 phrases; correct: 4926.
accuracy:  97.54%; precision:  85.86%; recall:  87.22%; FB1:  86.53
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              LOC: precision:  89.35%; recall:  91.01%; FB1:  90.17  1699
             MISC: precision:  70.14%; recall:  72.93%; FB1:  71.51  730
              ORG: precisi

## Conditional Random Fields (+2% Course Grade - optional extra credit)

Now we are ready to add a CRF layer to the `CharacterLSTMTagger`.  To train the model, implement `conditional_log_likelihood`, using the score (unnormalized log probability) of the gold sequence, in addition to the partition function, $Z(X)$, which is computed using the forward algorithm.  Then, you can simply use Pytorch's automatic differentiation to compute gradients by running backpropagation through the computation graph of the dynamic program (this should be very simple, so long as you are able to correctly implement the forward algorithm using a computation graph that is supported by PyTorch).  This approach to computing gradients for CRFs is discussed in Section 7.5.3 of the [Eisenstein Book](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)

You will also need to implement the Viterbi algorithm for inference during decoding.

After including CRF training and Viterbi decoding, you should be getting about **92 F1 / 88 F1 on the dev and test set**, respectively.

---




**IMPORTANT:** Note that training will be substantially slower this time - depending on the efficiency of your implementation, it could take about 5 minutes per epoch (e.g. 50 minutes for 10 iterations).  It is recommended to start out training on a single batch of data (and testing on this same batch), so that you can quickly debug, making sure your model can memorize the labels on a single batch, and then optimize your code.  Once you are fairly confident your code is working properly, then you can train using the full dataset.  We have provided a (commented out) line of code to switch between training on a single batch and the full dataset below.

**Hint #1:** While debugging your implementation of the Forward algorithm it is helpful to look at the loss during training.  The loss should never be less than zero (the log-likelihood should always be negative).

**Hint #2:** To sum log-probabilities in a numerically stable way at the end of the Forward algorithm, you will want to use [`torch.logsumexp`](https://pytorch.org/docs/stable/generated/torch.logsumexp.html).

**Fill in the functions marked as `TODO` in the code block below. Please make your code changes only within the given commented block (#####).**

In [ ]:
import torch.nn.functional as F

#####################################################################################
#TODO: Add imports if needed.

#####################################################################################

class LSTM_CRFtagger(CharLSTMtagger):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10):
        super(LSTM_CRFtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID, DIM_CHAR_EMB=DIM_CHAR_EMB)
        #####################################################################################
        #TODO: Initialize parameters.

        #####################################################################################

    def gold_score(self, lstm_scores, Y):
        #####################################################################################
        #TODO: compute score of gold sequence Y (unnormalized conditional log-probability)


        #####################################################################################


    #Forward algorithm for a single sentence
    #Efficiency will eventually be important here.  We recommend you start by
    #training on a single batch and make sure your code can memorize the
    #training data.  Then you can go back and re-write the inner loop using
    #tensor operations to speed things up.
    def forward_algorithm(self, lstm_scores, sLen):
        #####################################################################################
        #TODO: compute partition function Z


        #####################################################################################

    def conditional_log_likelihood(self, sentences, tags, train=True):
        #####################################################################################
        #TODO: compute conditional log likelihood of Y (use forward_algorithm and gold_score)

        #####################################################################################

    def viterbi(self, lstm_scores, sLen):
        #####################################################################################
        #TODO: Implement the Viterbi algorithm

        #####################################################################################

    #Computes Viterbi sequences on a batch of data.
    def viterbi_batch(self, sentences):
        viterbiSeqs = []
        (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
        lstm_scores = self.forward(X.cuda(), X_char.cuda(), train=True)
        for s in range(len(sentences)):
            (viterbiSeq, ll) = self.viterbi(lstm_scores[s], len(sentences[s]))
            viterbiSeqs.append(viterbiSeq)
        return viterbiSeqs


    def forward(self, X, X_char, train=False):
        #####################################################################################
        #TODO: Implement the forward computation.

        #####################################################################################


    def print_predictions(self, words, tags):
        Y_pred = self.inference(words)
        for i in range(len(words)):
            print("----------------------------")
            print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
            print("Predicted:\t", [Y_pred[i][j] for j in range(len(words[i]))])
            print("Gold:\t\t", tags[i])

    #Need to use Viterbi this time.
    def inference(self, sentences, viterbi=True):
        pred = self.viterbi_batch(sentences)
        return [[i2tag[pred[i][j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

lstm_crf = LSTM_CRFtagger(DIM_EMB=300).cuda()
# print(lstm_crf.conditional_log_likelihood(sentences_dev[0:5], tags_dev[0:5]))

In [ ]:
# This is a cell for debugging, feel free to change it as you like
print(lstm_crf.conditional_log_likelihood(sentences_dev[0:5], tags_dev[0:5]))

In [ ]:
#CharLSTM-CRF Training

#####################################################################################
# TODO: Add imports if needed.

#####################################################################################

#Get CoNLL evaluation script
os.system('wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl')

def train_crf_lstm(sentences, tags, lstm):
    #####################################################################################
    #TODO: initialize optimizer and hyperparameters.

    #####################################################################################
    for epoch in range(nEpochs):
        totalLoss = 0.0
        lstm.train()

        #Shuffle the sentences
        (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
        for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
            #####################################################################################
            #TODO: Implement gradient update on a batch of data.

            #####################################################################################



        print(f"loss on epoch {epoch} = {totalLoss}")
        lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
        print('conlleval:')
        print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

        if epoch % 5 == 0:
            lstm.eval()
            s = random.sample(range(50), 5)
            lstm.print_predictions([sentences_train[i] for i in s], [tags_train[i] for i in s])   #Print predictions on train data (useful for debugging)

crf_lstm = LSTM_CRFtagger(DIM_HID=500, DIM_EMB=300, DIM_CHAR_EMB=30).cuda()
train_crf_lstm(sentences_train, tags_train, crf_lstm)             #Train on the full dataset
# train_crf_lstm(sentences_train[0:50], tags_train[0:50], crf_lstm)   #Train only the first batch (use this during development/debugging)

In [ ]:
crf_lstm.eval()
crf_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm_crf.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm_crf.txt | perl conlleval.pl -d "\t"

## Gradescope

This is the end. Congratulations!  

Now, follow the steps below to submit your homework in [Gradescope](https://www.gradescope.com/courses/939466):

1. Rename this ipynb file to 'CS4650_p2_GTusername.ipynb'. We recommend ensuring you have removed any extraneous cells & print statements, clearing all outputs, and using the Runtime --> Run all tool to make sure all output is update to date. Additionally, leaving comments in your code to help us understand your operations will assist the teaching staff in grading. It is not a requirement, but is recommended.
2. Click on the menu 'File' --> 'Download' --> 'Download .py'.
3. Click on the menu 'File' --> 'Download' --> 'Download .ipynb'.
4. Download the notebook as a .pdf document. Make sure the training and evaluation output are captured so we can see how the loss and accuracy changes while training.
5. Download the predictions from Colab by clicking the folder icon on the left and finding them under Files, including 'test_pred_lstm.txt', 'test_pred_cnn_lstm.txt', and 'test_pred_cnn_lstm_crf.txt' (optional).
5. Upload all 5 or 6 files to GradeScope:
> CS4650_p2_GTusername.ipynb
>
> CS4650_p2_GTusername.py
>
> CS4650_p2_GTusername.pdf
>
> test_pred_lstm.txt
>
> test_pred_cnn_lstm.txt
>
> test_pred_cnn_lstm_crf.txt (optional)


**Please make sure your implementation meets the accuracy requirements to get full credit.**

**Please make sure that you name the files as specified above. You will be able to see the test set accuracy for your predictions on leaderboard. However, the final score will be assigned later based on accuracy in the notebook / PDF and implementation.**

You can submit multiple times before the deadline and choose the submission which you want to be graded by going to `Submission History` on gradescope.
